## OPTIMIZATION 2 - added 2 hidden layers + decreased epochs + decreased APPLICATION_TYPE bins + decreased CLASSIFICATION bins

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis='columns', inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Look at ASK_AMT value counts for binning
application_df['ASK_AMT'].value_counts()

5000       25398
10478          3
15583          3
6725           3
63981          3
           ...  
772556         1
70103          1
27096          1
25049          1
1138700        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [4]:
# Create array pf ask_amt values
ask_amt_values = application_df['ASK_AMT'].unique()
ask_amt_values

array([    5000,   108590,     6692, ...,     6713,    20197, 36500179],
      dtype=int64)

In [5]:
# Choose a cutoff value and create a list of ask_amt to be replaced
# use the variable name `ask_amt_bin`
ask_amt_bin = ask_amt_values[ask_amt_values > 5000]

# Replace in dataframe
for bin in ask_amt_bin:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(bin,"Other")
    
# Check to make sure binning was successful
application_df['ASK_AMT'].value_counts()

5000     25398
Other     8901
Name: ASK_AMT, dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T8', 'T7', 'T10', 'T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
value_counts = application_df['CLASSIFICATION'].value_counts()
value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C3700        1
C8210        1
C2190        1
C4200        1
C1900        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(value_counts[value_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_5000,ASK_AMT_Other
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,1,0
3,1,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
4,1,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Fourth hidden layer
nn3.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3360      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 30)                930       
                                                                 
 dense_3 (Dense)             (None, 30)                930       
                                                                 
 dense_4 (Dense)             (None, 1)                 31        
                                                                 
Total params: 7,681
Trainable params: 7,681
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=75)

Epoch 1/75
804/804 [==============================] - 1s 834us/step - loss: 0.5732 - accuracy: 0.7179
Epoch 2/75
804/804 [==============================] - 1s 787us/step - loss: 0.5561 - accuracy: 0.7290
Epoch 3/75
804/804 [==============================] - 1s 848us/step - loss: 0.5535 - accuracy: 0.7318
Epoch 4/75
804/804 [==============================] - 1s 848us/step - loss: 0.5521 - accuracy: 0.7327
Epoch 5/75
804/804 [==============================] - 1s 853us/step - loss: 0.5514 - accuracy: 0.7330
Epoch 6/75
804/804 [==============================] - 1s 875us/step - loss: 0.5505 - accuracy: 0.7322
Epoch 7/75
804/804 [==============================] - 1s 874us/step - loss: 0.5497 - accuracy: 0.7342
Epoch 8/75
804/804 [==============================] - 1s 842us/step - loss: 0.5486 - accuracy: 0.7343
Epoch 9/75
804/804 [==============================] - 1s 858us/step - loss: 0.5476 - accuracy: 0.7327
Epoch 10/75
804/804 [==============================] - 1s 880us/step - loss: 0.547

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5788 - accuracy: 0.7236 - 253ms/epoch - 944us/step
Loss: 0.5787608027458191, Accuracy: 0.7236151695251465


In [17]:
# Export our model to HDF5 file
nn3.save("AlphabetSoupCharity_Optimization_2.h5")